In [11]:
import numpy as np
from datetime import datetime, timedelta
from time import time
import os 
os.getcwd()

'/mnt/data/work/FourCastNet/data_nowcasting'

In [12]:
start_date = datetime(2010, 1, 11)
end_date = datetime(2010, 1, 31)


In [13]:
value = []
date_list = []
for i in range((end_date-start_date).days):
    t0 = time()
    c_date = start_date+timedelta(i)
    filename = '/home/cxt/data/work/Data/WACCM/{}/Density/FWSD_2010.cam.h1.{}-00000.Density.2-5km.npz'.format(c_date.year, c_date.strftime('%Y-%m-%d'))

    seconds = np.arange(0, 24*60*60, 60*30)
    date_i = np.array([c_date+timedelta(0, int(seconds[i])) for i in range(seconds.size)])

    if os.path.exists(filename) is True:
        value.append(np.array(np.load(filename)['value'].astype('float32'))[:, -1])      
        date_list.append(date_i)
        t1 = time()
        print('time usage:{}   finish:{}'.format(t1-t0, filename))
    else:
        print('not found: {}'.format(filename))


pass

time usage:0.3056221008300781   finish:/home/cxt/data/work/Data/WACCM/2010/Density/FWSD_2010.cam.h1.2010-01-11-00000.Density.2-5km.npz
time usage:0.30362558364868164   finish:/home/cxt/data/work/Data/WACCM/2010/Density/FWSD_2010.cam.h1.2010-01-12-00000.Density.2-5km.npz
time usage:0.29862451553344727   finish:/home/cxt/data/work/Data/WACCM/2010/Density/FWSD_2010.cam.h1.2010-01-13-00000.Density.2-5km.npz
time usage:0.3084716796875   finish:/home/cxt/data/work/Data/WACCM/2010/Density/FWSD_2010.cam.h1.2010-01-14-00000.Density.2-5km.npz
time usage:0.31099820137023926   finish:/home/cxt/data/work/Data/WACCM/2010/Density/FWSD_2010.cam.h1.2010-01-15-00000.Density.2-5km.npz
time usage:0.30647873878479004   finish:/home/cxt/data/work/Data/WACCM/2010/Density/FWSD_2010.cam.h1.2010-01-16-00000.Density.2-5km.npz
time usage:0.3013474941253662   finish:/home/cxt/data/work/Data/WACCM/2010/Density/FWSD_2010.cam.h1.2010-01-17-00000.Density.2-5km.npz
time usage:0.30133628845214844   finish:/home/cxt/data

In [14]:
ndensity = np.concatenate(value, axis=0)
ndensity_log = np.log(ndensity)
ndensity = ndensity_log-np.mean(ndensity_log)
date_list = np.hstack(date_list)


In [30]:
import os
print(os.getcwd())
os.makedirs('train', exist_ok=True)
os.makedirs('test', exist_ok=True)


/mnt/data/work/FourCastNet/data_nowcasting


In [31]:
data = ndensity.reshape((-1, 6)+ndensity.shape[1:])
train_mask = data.shape[0]//6*5

In [44]:
import h5py
with h5py.File('train/2010.h5', 'w') as f:
    f.create_dataset("fields", data=data[:train_mask])
    
with h5py.File('test/2010.h5', 'w') as f:
    f.create_dataset("fields", data=data[train_mask:])
